In [ ]:
## this code reads in the sound_labels_per_clip csv and performs two t-tests:
## one for amygdala peaks versus troughs, and one for MD peaks versus troughs. 
## (it also corrects for multiple comparisons using Holm-Bonferroni)
## it also outputs two graphs in light blue, one for amygdala and one for MD. 
## you can adapt "Speech" to calculate the t-tests for any other sound. 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import matplotlib as mpl          # do this right after the usual imports

# --- one-time global style tweaks -----------------------
mpl.rcParams.update({
    "figure.facecolor": "white",   # whole figure background
    "axes.facecolor":   "white",   # axes panel background
    "axes.grid": False             # kill the default grid
})
# --------------------------------------------------------


# ------------------------------------------------------------------
# 1.  Read the data once
# ------------------------------------------------------------------
df = pd.read_csv("sound_labels_per_clip.csv")

# ------------------------------------------------------------------
# 2.  Define the two Speech-count contrasts we care about
# ------------------------------------------------------------------
comparisons = [("AMY-HI", "AMY-LO"),
               ("MD-HI",  "MD-LO")]

# Pretty names for plotting
display = {"AMY-HI": "AM_high",
           "AMY-LO": "AM_low",
           "MD-HI":  "MD_high",
           "MD-LO":  "MD_low"}

# ------------------------------------------------------------------
# 3.  Run both t-tests and collect results
# ------------------------------------------------------------------
results, raw_pvals = [], []

for hi, lo in comparisons:
    subset = df[df["AM_Label"].isin([hi, lo])]

    # descriptive stats for Speech counts
    desc = subset.groupby("AM_Label")["Speech"].agg(["mean", "sem"]).reindex([hi, lo])

    hi_vals = subset[subset["AM_Label"] == hi]["Speech"]
    lo_vals = subset[subset["AM_Label"] == lo]["Speech"]
    t_stat, p_val = ttest_ind(hi_vals, lo_vals, equal_var=True, nan_policy="omit")

    results.append({"hi": hi, "lo": lo,
                    "desc": desc,
                    "t": t_stat, "p_raw": p_val})
    raw_pvals.append(p_val)

# ------------------------------------------------------------------
# 4.  Correct the p-values (Holm by default)
# ------------------------------------------------------------------
try:
    from statsmodels.stats.multitest import multipletests
    method = "holm"              # change to 'bonferroni', 'fdr_bh', …
    _, p_corr, _, _ = multipletests(raw_pvals, method=method)
except ImportError:
    method = "bonferroni"
    m = len(raw_pvals)
    p_corr = [min(p * m, 1.0) for p in raw_pvals]

for r, pc in zip(results, p_corr):
    r["p_corr"] = pc

print(f"Using {method} correction for {len(raw_pvals)} Speech-count contrasts\n")
for r in results:
    print(f"{r['hi']} vs {r['lo']}: "
          f"t = {r['t']:.3f},  p_raw = {r['p_raw']:.4g},  p_corr = {r['p_corr']:.4g}")

# ------------------------------------------------------------------
# 5.  Helper for significance stars
# ------------------------------------------------------------------
def stars(p):
    if p < 0.001: return "***"
    if p < 0.01:  return "**"
    if p < 0.05:  return "*"
    return "ns"

# ------------------------------------------------------------------
# 6.  Plot each comparison
# ------------------------------------------------------------------
for r in results:
    hi, lo = r["hi"], r["lo"]
    desc   = r["desc"]

    fig, ax = plt.subplots(figsize=(8, 6))
    x = np.arange(2)

    ax.bar(x,
           desc["mean"],
           yerr=desc["sem"],
           width=0.6,
           #color="#A5B299",
           color='#4194CB',
           capsize=5,
           edgecolor="black")
    ax.axhline(0, color="gray", linestyle="--", linewidth=1)

    # bracket & stars
    offset = (desc["mean"] + desc["sem"]).max() * 0.10
    h      = offset * 0.25
    y_base = max(desc.loc[hi, "mean"] + desc.loc[hi, "sem"],
                 desc.loc[lo, "mean"] + desc.loc[lo, "sem"])
    y = y_base + offset

    ax.plot([0, 0, 1, 1], [y, y+h, y+h, y], color="black", lw=1.2)
    ax.text(0.5, y + h + offset * 0.05, stars(r["p_corr"]),
            ha="center", va="bottom", fontsize=12)

    # tick labels & axis labels
    ax.set_xticks(x)
    ax.set_xticklabels([display[hi], display[lo]], fontsize=12)
    ax.set_ylabel("Mean Speech count", fontsize=14)
    ax.set_xlabel("Group", fontsize=14)

    ax.set_title(f"Mean Speech Count ± SEM: {display[hi]} vs {display[lo]}\n"
                 f"(p = {r['p_corr']:.4g}, {method}-corrected)",
                 fontsize=16)
    plt.tight_layout()
    plt.show()
